
# Structural Control Flow: Factory and Strategy Patterns

## Introduction

In this notebook, we will learn how to replace **procedural control flow** with **object-oriented design patterns**:
- **Factory Pattern**: Encapsulates object creation to improve maintainability.
- **Strategy Pattern**: Encapsulates business logic variations to improve flexibility.

This approach helps us eliminate long `if-elif` chains and makes our code **scalable and easy to extend**.

---

## Step 1: The Problem with Procedural Code

Consider this basic function to calculate tariff rates:


In [1]:
def calculate_tariff(base_cost, country):
    if country == "Mexico":
        tariff_rate = 0.25
    elif country == "China":
        tariff_rate = 0.10
    else:
        tariff_rate = 0.05

    return base_cost * (1 + tariff_rate)

# Test case
print(f"Total cost (Mexico): ${calculate_tariff(100, 'Mexico'):.2f}")
print(f"Total cost (China): ${calculate_tariff(100, 'China'):.2f}")
print(f"Total cost (Other): ${calculate_tariff(100, 'Canada'):.2f}")

Total cost (Mexico): $125.00
Total cost (China): $110.00
Total cost (Other): $105.00


### 🚨 Problems with This Approach
1. **Hard to Maintain** – Adding a new country requires modifying this function.
2. **Violates Open-Closed Principle** – We should extend functionality without modifying existing code.
3. **Difficult to Test and Reuse** – Business rules are mixed with calculations.


## Step 2: What is the Factory Pattern? Where Does it Come From?

### 🎭 **What is the Factory Pattern?**
The **Factory Pattern** is a design pattern that provides a way to create objects **without specifying their exact class**. Instead of directly instantiating objects using `new` or `ClassName()`, we use a **factory method** that handles object creation.


In [2]:
class TariffStrategyFactory:
    strategies = {
        "Mexico": MexicanTariffStrategy(),
        "China": ChineseTariffStrategy(),
    }

    @staticmethod
    def get_strategy(country: str):
        return TariffStrategyFactory.strategies.get(country, DefaultTariffStrategy())

# Usage without instantiating the factory
strategy = TariffStrategyFactory.get_strategy("Mexico")
print(strategy.calculate_tariff(100))

NameError: name 'MexicanTariffStrategy' is not defined

## Step 3: Understanding the Static Method Decorator

In Python, the **`@staticmethod` decorator** is used to define a method that does not operate on an instance of the class (i.e., it does not access `self`).

### **How It Works**:
- Static methods belong to the class rather than an instance of the class.
- They are useful for utility functions or logic that is related to the class but does not need instance-specific data.


## Step 4: What is the Strategy Pattern?

### 🎯 **What is the Strategy Pattern?**
The **Strategy Pattern** is a behavioral design pattern that allows a family of algorithms (strategies) to be defined and used interchangeably. Instead of embedding multiple algorithms in a single class, the Strategy Pattern delegates the responsibility to different classes, improving flexibility and maintainability.


In [3]:
from abc import ABC, abstractmethod

class TariffStrategy(ABC):
    @abstractmethod
    def calculate_tariff(self, base_cost: float) -> float:
        pass

class MexicanTariffStrategy(TariffStrategy):
    def calculate_tariff(self, base_cost: float) -> float:
        return base_cost * 1.25

class ChineseTariffStrategy(TariffStrategy):
    def calculate_tariff(self, base_cost: float) -> float:
        return base_cost * 1.10

class DefaultTariffStrategy(TariffStrategy):
    def calculate_tariff(self, base_cost: float) -> float:
        return base_cost * 1.05

## Step 5: Supply Chain Example (Inspired by the Automotive Diagram)

### 🚗 **Using Patterns for a Complex Supply Chain**
Let’s model a supply chain where parts move through multiple locations, and tariffs or logistics costs apply based on the region.


In [6]:
class RegionTariffStrategy(TariffStrategy):
    def __init__(self, tariff_rate: float, logistics_cost: float):
        self.tariff_rate = tariff_rate
        self.logistics_cost = logistics_cost

    def calculate_tariff(self, base_cost: float) -> float:
        return base_cost * (1 + self.tariff_rate) + self.logistics_cost

class SupplyChainTariffFactory:
    regions = {
        "USA": RegionTariffStrategy(0.10, 50),
        "Mexico": RegionTariffStrategy(0.20, 30),
        "Canada": RegionTariffStrategy(0.15, 40),
    }

    @staticmethod
    def get_strategy(region: str):
        return SupplyChainTariffFactory.regions.get(region, RegionTariffStrategy(0.05, 20))

class SupplyChainPart:
    def __init__(self, name: str, base_cost: float, region: str):
        self.name = name
        self.base_cost = base_cost
        self.region = region
        self.tariff_strategy = SupplyChainTariffFactory.get_strategy(region)

    def calculate_total_cost(self) -> float:
        return self.tariff_strategy.calculate_tariff(self.base_cost)

    def print_details(self):
        total_cost = self.calculate_total_cost()
        print(f"Part: {self.name} | Region: {self.region} | Total Cost: ${total_cost:.2f}")

# Example Usage
capacitor = SupplyChainPart("Capacitor", 200, "Mexico")
capacitor.print_details()
actuator = SupplyChainPart("Actuator", 500, "Canada")
actuator.print_details()

Part: Capacitor | Region: Mexico | Total Cost: $270.00
Part: Actuator | Region: Canada | Total Cost: $615.00


Okay this next part is the Beer Game Supply Chain example from class...

In [8]:
class OrderingStategy:
  def compute_order(self, external_order, backlog, inventory, target_inventory):
    '''
    the logic of how each agent will act
    '''
    order = external_order + backlog + (target_inventory - inventory)
    return max(order, 0)

In [9]:
class SupplyChainAgent:
  def __init__(self, name, country, lead_time, initial_inventory, target_inventory, ordering_strategy, is_manufacturer = False):
    self.name = name
    self.country = country
    self.lead_time = lead_time
    self.inventory = initial_inventory
    self.target_inventory = target_inventory
    self.ordering_strategy = ordering_strategy # notice this is the input argument
    self.is_manufacturer = is_manufacturer

    self.backlog = 0
    self.incoming_shipments = []

    self.order_from_downstream = 0
    self.order_from_upstream = 0

    self.upstream = None
    self.downstream = None



In [11]:
class SupplyChainAgentFactory:
  def __init__(self, ordering_stategy):
    self.ordering_strategy = ordering_stategy

  def create_agent(self, name, country, lead_time, initial_inventory, target_inventory, is_manufacturer = False):
    return SupplyChainAgent(
        name = name,
        country = country,
        lead_time = lead_time,
        initial_inventory = initial_inventory,
        target_inventory = target_inventory,
        ordering_strategy = self.ordering_strategy, # notice this is not an input argument, it's an attribute from the ordering constructor
        is_manufacturer = is_manufacturer
    )

In [12]:
class Shipment:
  def __init__(self, quantity, delay, source, destination):
    self.quantity = quantity # should be 1
    self.delay = delay
    self.source = source
    self.destination = destination